## Getting Data Using Kaggle API 

In [98]:
!kaggle datasets list -s 'coffee'

ref                                                           title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
michals22/coffee-dataset                                      Coffee dataset                                     24KB  2022-12-15 20:02:12          14454        215  1.0              
ihelon/coffee-sales                                           Coffee Sales                                       13KB  2024-08-01 07:55:34           9065        146  1.0              
ahmedabbas757/coffee-sales                                    Coffee Shop Sales                                   8MB  2023-12-01 19:04:06           7011         82  0.88235295       
schmoyote/coffee-reviews-dataset                              Coffee Reviews Dat

In [99]:
!kaggle datasets download -d ahmedabbas757/coffee-sales

Dataset URL: https://www.kaggle.com/datasets/ahmedabbas757/coffee-sales
License(s): GNU Lesser General Public License 3.0




  0%|          | 0.00/8.23M [00:00<?, ?B/s]
 12%|█▏        | 1.00M/8.23M [00:01<00:10, 714kB/s]
 24%|██▍       | 2.00M/8.23M [00:02<00:06, 1.01MB/s]
 36%|███▋      | 3.00M/8.23M [00:02<00:04, 1.15MB/s]
 49%|████▊     | 4.00M/8.23M [00:03<00:03, 1.22MB/s]
 61%|██████    | 5.00M/8.23M [00:04<00:02, 1.17MB/s]
 73%|███████▎  | 6.00M/8.23M [00:05<00:01, 1.23MB/s]
 85%|████████▌ | 7.00M/8.23M [00:06<00:01, 1.27MB/s]
 97%|█████████▋| 8.00M/8.23M [00:07<00:00, 1.28MB/s]
100%|██████████| 8.23M/8.23M [00:07<00:00, 1.30MB/s]
100%|██████████| 8.23M/8.23M [00:07<00:00, 1.20MB/s]


## Import Data

In [15]:
import pandas as pd 

In [16]:
df = pd.read_excel('./coffee-sales/Coffee Shop Sales.xlsx')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149116 entries, 0 to 149115
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   transaction_id    149116 non-null  int64         
 1   transaction_date  149116 non-null  datetime64[ns]
 2   transaction_time  149116 non-null  object        
 3   transaction_qty   149116 non-null  int64         
 4   store_id          149116 non-null  int64         
 5   store_location    149116 non-null  object        
 6   product_id        149116 non-null  int64         
 7   unit_price        149116 non-null  float64       
 8   product_category  149116 non-null  object        
 9   product_type      149116 non-null  object        
 10  product_detail    149116 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 12.5+ MB


## Data Cleaning

In [18]:
# total sales
df['total_payment'] = df['transaction_qty'] * df['unit_price']

In [19]:
# concat transaction date and transaction time 
df['transaction_datetime'] = pd.to_datetime(df['transaction_date']) + pd.to_timedelta(df['transaction_time'].astype(str))

df = df.drop(columns=['transaction_date', 'transaction_time'], axis=1)

In [20]:
# check nan data
df.isnull().sum()

transaction_id          0
transaction_qty         0
store_id                0
store_location          0
product_id              0
unit_price              0
product_category        0
product_type            0
product_detail          0
total_payment           0
transaction_datetime    0
dtype: int64

In [21]:
df['product_detail'] = df['product_detail'].str.replace('Rg', "Regular")
df['product_detail'] = df['product_detail'].str.replace('Lg', "Large")
df['product_detail'] = df['product_detail'].str.replace('Sm', "Small")
df['product_detail'] = df['product_detail'].str.replace('syrup', "Syrup")
df['product_detail'] = df['product_detail'].str.replace('shot', "Shot")
df['product_detail'] = df['product_detail'].str.replace('chocolate', "Chocolate")
df['product_detail'] = df['product_detail'].str.replace('mug', "Mug")

In [22]:
df = df.replace({'Gourmet brewed coffee' : 'Gourmet Brewed Coffee',
            'Brewed Chai tea' : 'Brewed Chai Tea',
            'Hot chocolate' : 'Hot Chocolate',
            'Drip coffee' : 'Drip Coffee', 
            'Brewed Black tea' : 'Brewed Black Tea',
            'Brewed Green tea' : 'Brewed Green Tea', 
            'Brewed herbal tea' : 'Brewed Herbal Tea', 
            'Organic brewed coffee' : 'Organic Brewed Coffee', 
            'Premium brewed coffee' : 'Premium Brewed Coffee', 
            'Regular syrup' : 'Regular Syrup',
            'Herbal tea' : 'Herbal Tea', 
            'Sugar free syrup' : 'Sugar Free Syrup',
            'Green beans' : 'Green Beans',
            'Green tea' : 'Green Tea', 
            'Black tea' : 'Black Tea', 
            'House blend Beans' : 'House Blend Beans',
            'Coffee beans' : 'Coffee Beans'})

In [23]:
#checking duplicate data
df['transaction_id'].duplicated().sum()

0

In [30]:
#changing range transaction id 
df['transaction_id'] = range(1, len(df)+1)

In [31]:
df_cleaned = df.to_csv('./df_cleaned.csv', index=False)

In [32]:
df = df.copy()

In [33]:
#separate data based on store location
lm_df = df[df['store_location'] == 'Lower Manhattan']
hk_df = df[df['store_location'] == "Hell's Kitchen"]
as_df = df[df['store_location'] == 'Astoria']

In [34]:
lm_df = lm_df.drop(columns=['store_id', 'store_location']).reset_index(drop=True)
hk_df = hk_df.drop(columns=['store_id', 'store_location']).reset_index(drop=True)
as_df = as_df.drop(columns=['store_id', 'store_location']).reset_index(drop=True)

In [35]:
lm_df['transaction_datetime'] = lm_df['transaction_datetime'].sort_values()
hk_df['transaction_datetime'] = hk_df['transaction_datetime'].sort_values()
as_df['transaction_datetime'] = as_df['transaction_datetime'].sort_values()

In [37]:
lm_df.to_csv('./lower_manhattan.csv', index=False)
hk_df.to_csv('./hell_kitchen.csv',index=False)
as_df.to_csv('./astoria.csv', index=False)